In [1]:
# because the valve samples are not continous, but rather they click with an interval between it makes 
# sense to look at a different type of modeling.
# first idea is to check if peaks occur at regular intervals. If not it might be an anomaly.
# second is to split the files by peak

# both need some form of peak detection.


In [13]:
import os
import librosa
import pandas as pd
import numpy as np

In [3]:
# get the parent directory
parent_dir = os.path.abspath('..')
test_dir = os.path.join(parent_dir, 'Testing')
data_dir = os.path.join(parent_dir, 'Data')
meta_dir = os.path.join(data_dir, 'metadata')
sour_dir = os.path.join(data_dir, 'source')
data_dir = os.path.join(parent_dir, 'EDA')

In [7]:
valve_subset = pd.read_csv(os.path.join(test_dir, 'valve_subset.csv'))

In [8]:
valve_subset

,file_no,machine,model,anomaly,file_name,file_rel_path,duration_s,fs_Hz,SNR_dB
0,13890,valve,0,1,00000041.wav,6_dB_valve\valve\id_00\abnormal\00000041.wav,10.0,16000,6
1,13954,valve,0,1,00000105.wav,6_dB_valve\valve\id_00\abnormal\00000105.wav,10.0,16000,6
2,13923,valve,0,1,00000074.wav,6_dB_valve\valve\id_00\abnormal\00000074.wav,10.0,16000,6
3,13873,valve,0,1,00000024.wav,6_dB_valve\valve\id_00\abnormal\00000024.wav,10.0,16000,6
4,13904,valve,0,1,00000055.wav,6_dB_valve\valve\id_00\abnormal\00000055.wav,10.0,16000,6
...,...,...,...,...,...,...,...,...,...
1595,17620,valve,6,0,00000593.wav,6_dB_valve\valve\id_06\normal\00000593.wav,10.0,16000,6
1596,17663,valve,6,0,00000636.wav,6_dB_valve\valve\id_06\normal\00000636.wav,10.0,16000,6
1597,17156,valve,6,0,00000129.wav,6_dB_valve\valve\id_06\normal\00000129.wav,10.0,16000,6
1598,17117,valve,6,0,00000090.wav,6_dB_valve\valve\id_06\normal\00000090.wav,10.0,16000,6


In [11]:
random_file = os.path.join(sour_dir, valve_subset.iloc[0]['file_rel_path'])


'C:\\Users\\aubin\\beCode\\anomaly\\AudioMalfunctionDetection\\Data\\source\\6_dB_valve\\valve\\id_00\\abnormal\\00000041.wav'

In [14]:
y, sr = librosa.load(random_file)

onset_env = librosa.onset.onset_strength(
    y=y, 
    sr=sr,
    hop_length=512,
    aggregate=np.median
)

peaks = librosa.util.peak_pick(onset_env, 3, 3, 3, 5, 0.5, 10)


In [15]:
peaks

array([ 22,  46,  70,  92, 116, 328, 375, 422], dtype=int64)

In [16]:
intervals = []
for idx, x in enumerate(peaks):
    try:
        intervals.append(peaks[idx+1] - peaks[idx])
    except:
        pass
    
intervals

[24, 24, 22, 24, 212, 47, 47]

In [24]:
normal_valve_subset = valve_subset.loc[valve_subset['anomaly'] == 0]
abnormal_valve_subset = valve_subset.loc[valve_subset['anomaly'] == 1]

In [19]:
normal_valve_subset

,file_no,machine,model,anomaly,file_name,file_rel_path,duration_s,fs_Hz,SNR_dB
100,14594,valve,0,0,00000626.wav,6_dB_valve\valve\id_00\normal\00000626.wav,10.0,16000,6
101,14261,valve,0,0,00000293.wav,6_dB_valve\valve\id_00\normal\00000293.wav,10.0,16000,6
102,14943,valve,0,0,00000975.wav,6_dB_valve\valve\id_00\normal\00000975.wav,10.0,16000,6
103,14525,valve,0,0,00000557.wav,6_dB_valve\valve\id_00\normal\00000557.wav,10.0,16000,6
104,14126,valve,0,0,00000158.wav,6_dB_valve\valve\id_00\normal\00000158.wav,10.0,16000,6
...,...,...,...,...,...,...,...,...,...
1595,17620,valve,6,0,00000593.wav,6_dB_valve\valve\id_06\normal\00000593.wav,10.0,16000,6
1596,17663,valve,6,0,00000636.wav,6_dB_valve\valve\id_06\normal\00000636.wav,10.0,16000,6
1597,17156,valve,6,0,00000129.wav,6_dB_valve\valve\id_06\normal\00000129.wav,10.0,16000,6
1598,17117,valve,6,0,00000090.wav,6_dB_valve\valve\id_06\normal\00000090.wav,10.0,16000,6


In [36]:
all_intervals_normal = []
all_intervals_abnormal = []

In [54]:
for idx, row in normal_valve_subset.iterrows():
    y, sr = librosa.load(os.path.join(sour_dir, row['file_rel_path']))

    onset_env = librosa.onset.onset_strength(
        y=y, 
        sr=sr,
        hop_length=512,
        aggregate=np.median
    )

    peaks_normal = librosa.util.peak_pick(onset_env, 3, 3, 3, 5, 0.5, 10)

    intervals = []
    for idx, x in enumerate(peaks_normal):
        try:
            intervals.append(peaks_normal[idx+1] - peaks_normal[idx])
        except:
            pass

    all_intervals_normal.append(intervals)

In [55]:
for idx, row in abnormal_valve_subset.iterrows():
    y, sr = librosa.load(os.path.join(sour_dir, row['file_rel_path']))

    onset_env = librosa.onset.onset_strength(
        y=y, 
        sr=sr,
        hop_length=512,
        aggregate=np.median
    )

    peaks_abnormal = librosa.util.peak_pick(onset_env, 3, 3, 3, 5, 0.5, 10)

    intervals = []
    for idx, x in enumerate(peaks_abnormal):
        try:
            intervals.append(peaks_abnormal[idx+1] - peaks_abnormal[idx])
        except:
            pass

    all_intervals_abnormal.append(intervals)

In [32]:
all_averages = []

In [33]:
for x in all_intervals_normal:
    average = sum(x) / len(x)
    all_averages.append(average)

In [38]:
all_averages_abn = []

for x in all_intervals_abnormal:
    average = sum(x) / len(x)
    all_averages_abn.append(average)

In [42]:
max(all_averages), min(all_averages)

(47.5, 18.727272727272727)

In [43]:
max(all_averages_abn), min(all_averages_abn)

(80.2, 20.142857142857142)

In [51]:
count = 0
total = 0
for x in all_averages_abn:
    total += 1
    if x > max(all_averages):
        count += 1
        
ratio = count / total

In [48]:
ratio

0.095

In [57]:
all_intervals_normal, all_intervals_abnormal

([[113, 22, 24, 23, 24, 23, 24, 23, 24],
  [24, 24, 23, 24, 23, 23, 23, 24, 24, 23, 24, 23, 23, 24, 23, 24, 23],
  [23, 24, 23, 24, 24, 23, 24, 108],
  [23, 23, 24, 24, 23, 23, 25, 211, 24],
  [23, 24, 23, 24, 23, 23, 24, 13, 11, 24, 24, 23, 17, 30, 24, 23, 24, 23, 23],
  [24, 23, 24, 23, 24, 23, 24, 187, 24, 24],
  [23, 25, 22, 25, 23, 15, 32, 24, 23, 24, 23, 24, 23, 13, 11, 24, 23, 23],
  [210, 25, 22, 21, 11, 16, 24, 23, 24],
  [24, 23, 23, 24, 23, 23, 24, 23, 24, 24, 23, 24, 24, 23, 24, 23, 23],
  [24, 24, 23, 24, 23, 24, 22, 24, 24, 24, 23, 23, 24, 24, 15, 32, 23, 24],
  [24, 24, 23, 24, 24, 23, 25],
  [24, 24, 23, 15, 32, 23, 24],
  [23, 24, 12, 14, 21, 45, 24, 85, 33, 13, 11, 18, 13, 16, 24, 23, 24],
  [23, 15, 32, 24, 23, 23, 24, 24, 23, 23, 13, 11, 24, 23, 24, 23, 24, 23],
  [23, 24, 23, 24, 24, 23, 24, 24, 23, 24, 23, 24, 24, 24, 23, 24, 24],
  [24, 213, 24, 23, 24, 22, 24, 23, 24],
  [48, 23, 24, 24, 23, 24, 23, 23, 23, 24, 23, 21, 26, 20, 27, 12, 12],
  [23, 24, 24, 23, 24,

In [49]:
# the last 5 cells show that you can easeily detect 10% of the malfunction of the valves simply 
# by checking for irregularities in the time interval between the opening and closing of the valve